In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# Langchain configuration
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# OpenAI API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Tavily API Key
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In this example, we use [Tavily](https://tavily.com/), a search API for LLMs, to search.


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-27', 'content': 'Current Weather for Popular Cities . San Francisco, CA 58 ° F Fair; Manhattan, NY warning 73 ° F Clear; Schiller Park, IL (60176) warning 76 ° F Mostly Cloudy; Boston, MA 65 ° F Cloudy ...'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719502931, 'localtime': '2024-06-27 8:42'}, 'current': {'last_updated_epoch': 1719502200, 'last_updated': '2024-06-27 08:30', 'temp_c': 14.4, 'temp_f': 57.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 231, 'wind_dir': 'SW', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 84, 'cloud': 100, 'feelslike_c': 14

# Use Language Model

In this example, we use OpenAI GPT model to use Tool.


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


model = ChatOpenAI()  # GPT-3.5-turbo by default

# Invoke the model with a message
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

Use `bind_tools` to give the language model knowledge of these tools.


In [4]:
model_with_tools = model.bind_tools(tools)

In [5]:
response = model_with_tools.invoke([HumanMessage(content="My name is Clement!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello Clement! How can I assist you today?
ToolCalls: []


In [6]:
response = model_with_tools.invoke(
    [HumanMessage(content="What's the weather in Taipei?")]
)
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Taipei'}, 'id': 'call_b1BPWyeipvROsaWVhZgGSaBL'}]


# Agent

To build an agent, we need to use [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph). It provides high level interface to construct an agent. Also, it is backed with low level logic so that we don't need to call `model.bind_tools` since it is already done under the hood.


In [7]:
from langgraph.prebuilt import create_react_agent

# Create the agent with the model and tools
agent_executor = create_react_agent(model, tools)

Let's see how it responds when there's no need to call a tool:


In [8]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='8a6b4ae5-a5d3-482f-81b6-c2fcfd1d2950'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-79a51e09-971c-4eb1-bb87-5405ab62ae0b-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

Let's see how it responds when there's a need to call a tool:


In [9]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='3c832ba5-5028-46ea-8950-cb00813450e1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hrPUkpkXs4CvRLqPBDnR5Oif', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f1712407-ce4e-47de-89fe-9f2971fe36e9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_hrPUkpkXs4CvRLqPBDnR5Oif'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109}),
 ToolMessage(content='[{"url": "https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-27", "content": "Current Weather for Popular Cities . San Francisco, CA 58 \\u00b0 F Fair; Manha

### Streaming

We can also use streaming to get the response in chunks.


In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather tomorrow in Taipei?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pwSDhbsU8qT2DBsuIjsvU6TR', 'function': {'arguments': '{"query":"weather tomorrow in Taipei"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 89, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-54f37656-d1df-4e75-8ef8-40c63d20ea53-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather tomorrow in Taipei'}, 'id': 'call_pwSDhbsU8qT2DBsuIjsvU6TR'}], usage_metadata={'input_tokens': 89, 'output_tokens': 21, 'total_tokens': 110})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Taipei\', \'region\': \\"T\'ai-pei\\", \'country\': \'Taiwan\', \'lat\': 25.04, \'lon\': 121.53, \'tz_id\': \'Asia/Taipei\', \'loca

To stream the response by **token**. We can use `astream_events` method.


In [12]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/Users/clement/Developer/ai/langchain-notebooks/.venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.timeanddate.com/weather/usa/san-francisco/hourly', 'content': 'Hour-by-Hour Forecast for San Francisco, California, USA. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 80 °F. Sunny. (Weather station: Hayward Air Terminal, USA). See more current weather.'}, {'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA', 'content': 'San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 4:13 pm PDT Jun 24, 2024. Forecast Valid: 7pm PDT Jun 24, 2024-6pm PDT Jul 1, 2024 . Forecast Discussion . Additional Resources. Radar & Satellite Image. Hourly Weather Forecast. ... Severe Weather ; Current Outlook Maps ; Drought ; Fire Weather ; Fronts/Precipitation Maps ...'}]
--
The| current| weather| in| San| Francisco|,| California| is| |80|°F| and| sunny|

# Memory

Agent is stateless so we can't store information between calls. To give it memory, we need to pass in a **checkpointer**. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

FYI, when using `RunnableWithMessageHistory`, we need to pass in `session_id`.


In [13]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [14]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c7c19e08-d99b-4ea9-9527-a3520ad18f72-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 108, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d4e85d2a-fe46-438d-82f3-65e434883718-0', usage_metadata={'input_tokens': 108, 'output_tokens': 14, 'total_tokens': 122})]}}
----
